In [ ]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob
import os
logging.basicConfig(level = logging.DEBUG)

# import os
# os.environ['KAGGLE_USERNAME'] = "vedula"
# os.environ['KAGGLE_KEY'] = "482a5c14ced45f63f3698eacb8fa0c62"

# import kaggle
# kaggle.api.dataset_download_files('nikhilpandey360/chest-xray-masks-and-labels/download', path='.', unzip=True)


preprocess = Transformation(
                "preprocess",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/preprocess.py",
                is_stageable=True
            )

data_split = Transformation(
                "data_split",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/data_split.py",
                is_stageable=True
            )

train_model = Transformation( 
                "train_model",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/train_model.py",
                is_stageable=True
            )


tc = TransformationCatalog().add_transformations(preprocess, data_split, train_model).write()

file_list = []
output_list = []

rc = ReplicaCatalog()

for file in glob.glob("./train_images/*.png"):
    f = file.replace("./train_images/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_images/").resolve() / f)
    
for file in glob.glob("./train_masks/*.png"):
    f = file.replace("./train_masks/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_masks/").resolve() / f)
    
for file in glob.glob("./test/*.png"):
    f = file.replace("./test/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./test/").resolve() / f)
    
rc.write()

for filename in glob.glob("./train_images/*.png"):
    f = filename.replace("./train_images/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))


for filename in glob.glob("./train_masks/*.png"):
    f = filename.replace("./train_masks/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

for filename in glob.glob("./test/*.png"):
    f = filename.replace("./test/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

        
wf = Workflow("preprocess")
    
job_preprocess = Job(preprocess)\
                    .add_inputs(*file_list)\
                    .add_outputs(*output_list)

fb = File("data_split.pkl")

job_data_split = Job(data_split)\
                    .add_inputs(*output_list)\
                    .add_outputs(fb)
    
fb1 = File("study_checkpoint.pkl")


job_train = Job(train_model)\
                    .add_inputs(*output_list, fb)\
                    .add_outputs(fb1)


wf.add_jobs(job_preprocess, job_data_split, job_train)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)  


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.10.06 09:35:05.103 UTC:
2020.10.06 09:35:05.108 UTC:   -----------------------------------------------------------------------
2020.10.06 09:35:05.114 UTC:   File for submitting this DAG to HTCondor           : preprocess-0.dag.condor.sub
2020.10.06 09:35:05.120 UTC:   Log of DAGMan debugging messages                 : preprocess-0.dag.dagman.out
2020.10.06 09:35:05.125 UTC:   Log of HTCond

[##################################----------------]  68.4% ..Running (Completed: 13, Queued: 0, Running: 1, Failed: 0)